In [ ]:
import polars as pl
import pandas as pd
import datetime 
from datetime import datetime as dt

import xmltodict
import json

In [ ]:
import pandas as pd

df=pd.read_csv(r'root_sample.csv')

In [ ]:
df.info(verbose=True)

In [ ]:
import prefect

In [ ]:
from src.utils import *

In [ ]:
import os
os.getcwd()

In [ ]:
df = pd.read_csv(r'pcb_2024_T01_T06.csv', nrows=1000)

In [ ]:
def conso_id(id1, id2):
    if id1 == "'":
        final_id = id2
    elif id2 == "'":
        final_id = id1
    else:
        if id1 == id2:
            final_id = id1
        else:
            final_id = id1+"_"+id2
    return final_id

In [ ]:
def get_log(row):
    full_message = xmltodict.parse(row)
    try:
        to_dict = full_message['MGResponse']['RI_Req_Output']['CreditHistory']
    except:
        to_dict = full_message['MGResponse']['CI_Req_Output']['CreditHistory']
    return str(to_dict)

In [ ]:
df.columns = ['id1', 'id2', 'name', 'address', 'log', 'process_date']

In [ ]:
df['process_date'] = df['process_date'].astype(str)

In [ ]:
df['cccd'] = df[['id1', 'id2']].apply(lambda 
                                      row: conso_id(row.id1, row.id2),
                                      axis=1
                                     )

In [ ]:
df['pcb'] = df['log'].apply(lambda row: get_log(row))

In [ ]:
from pip._vendor.rich.progress import (
    Progress, SpinnerColumn, TimeElapsedColumn
)

def parse_data(process_name, df_in, field, id_lst):
    try:
        df_holder = []
        total_rows = len(df_in)

        with Progress(SpinnerColumn(), *Progress.get_default_columns(), TimeElapsedColumn(), transient=False) as progress:
            task = progress.add_task(f"Parsing {process_name}", total=total_rows)
            def process_row(row):    
                explode_nest(df_holder, id_lst, row, row[field[0]])
                progress.update(task, advance=1)
                
            df_in[id_lst + field].apply(process_row, axis=1)
            df_out = pd.concat(df_holder)
            df_out = df_out.replace('null', np.nan)
    except KeyError:
        df_out = pd.DataFrame()
    return df_out

def gen_id_level2(df):
    return df['cccd'].astype(str) + '_' + df['CommonData.CBContractCode']

def treat_dtype(df, col):
    try:
        df[col] = df[col].astype(str)
    except KeyError:
        pass

In [ ]:
field = ['pcb']
id_col_list = ['cccd', 'process_date']

df_root = parse_data('root', df, field, id_col_list)

In [ ]:
dtype_treat_lst = ['Contract.Instalments.GrantedContract',
                  'Contract.Cards.GrantedContract',
                  'Contract.NonInstalments.GrantedContract',
                  'Contract.Instalments.NotGrantedContract',
                  'Contract.Cards.NotGrantedContract',
                  'Contract.NonInstalments.NotGrantedContract']
for i in dtype_treat_lst:
    treat_dtype(df_root, i)

In [ ]:
id_col_list = ['cccd', 'process_date']

contract_level_noninstall = parse_data('nonins', df_root, ['Contract.NonInstalments.GrantedContract'], id_col_list)
contract_level_install = parse_data('ins', df_root, ['Contract.Instalments.GrantedContract'], id_col_list)
contract_level_card = parse_data('card', df_root, ['Contract.Cards.GrantedContract'], id_col_list)

not_grant_install = parse_data('ng_ins', df_root, ['Contract.Instalments.NotGrantedContract'], id_col_list)
not_grant_noninstall = parse_data('ng_nonins', df_root, ['Contract.NonInstalments.NotGrantedContract'], id_col_list)
not_grant_card = parse_data('ng_card', df_root, ['Contract.Cards.NotGrantedContract'], id_col_list)

In [ ]:
contract_level_noninstall['loan_code_lv2'] = gen_id_level2(contract_level_noninstall)
contract_level_install['loan_code_lv2'] = gen_id_level2(contract_level_install)
contract_level_card['loan_code_lv2'] = gen_id_level2(contract_level_card)

treat_dtype(contract_level_install, 'Profiles')
treat_dtype(contract_level_install, 'InstGuarantees')
treat_dtype(contract_level_card, 'Profiles')
treat_dtype(contract_level_card, 'CardsGuarantees')
treat_dtype(contract_level_noninstall, 'Profiles')

In [ ]:
contract_level_card = contract_level_card[~contract_level_card['loan_code_lv2'].isna()]
contract_level_noninstall = contract_level_noninstall[~contract_level_noninstall['loan_code_lv2'].isna()]
contract_level_install = contract_level_install[~contract_level_install['loan_code_lv2'].isna()]  

In [ ]:
field = ['Profiles']
id_col_list = ['loan_code_lv2', 'cccd', 'CommonData.CBContractCode', 'process_date']

ts_card = parse_data('ts_card', contract_level_card, field, id_col_list)
ts_install = parse_data('ts_ins', contract_level_install, field, id_col_list)
ts_noninstall = parse_data('ts_nonins', contract_level_noninstall, field, id_col_list)

In [ ]:
ts_card

In [ ]:
df_root.head()